In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install torch datasets scikit-learn matplotlib
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 64.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's d

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import nltk
import pandas as pd

torch.manual_seed(2)

In [3]:
from datasets import load_dataset
from datasets import load_dataset_builder

dataset = load_dataset("imomayiz/darija-english", 'sentences')

dataset.set_format(type='pandas', output_all_columns=True)
dataset = dataset['sentences'][:]
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/348 [00:00<?, ?B/s]

sentences.csv:   0%|          | 0.00/6.34M [00:00<?, ?B/s]

Generating sentences split: 0 examples [00:00, ? examples/s]

,darija,eng,darija_ar
0,"homa mkhbbyin chi haja, ana mti99en!","They're hiding something, I'm sure!","هوما مخبّيين شي حاجة, أنا متيقّن!"
1,bayna homa tay7awlo ib9aw mbrrdin.,It's obvious they're trying to keep their cool.,باينا هوما تايحاولو إبقاو مبرّدين.
2,loTilat mabaynach fihom mori7in bzzaf.,the hotels don't seem very comfortable.,لوطيلات مابايناش فيهوم موريحين بزّاف.
3,ghaliban ghayjrriw 3lih mn lkhdma!,he is probably about to be laid off by head of...,غاليبان غايجرّيو عليه من لخدما!
4,Tab3an rah mkta2eb!,of course he's depressive!,طابعان راه مكتاءب!
...,...,...,...
87780,galou bli hadchi likayw9a3 rah khatir,None,ڭالو بلي هادشي ليكايوقاع راه خاتير
87781,kolchi na3as ana 3asa 3lihom,None,كولشي ناعاس أنا عاسا عليهوم
87782,ydi t9ila 3liya kanhas brasi m3asba,None,يدي تقيلا علييا كانهاس براسي معاسبا
87783,hadchi liw9ah mouakharn makhlach 3andi tarkiz,None,هادشي ليوقاه مواخارن ماخلاش عاندي تاركيز


In [4]:
df = dataset.dropna()
df

,darija,eng,darija_ar
0,"homa mkhbbyin chi haja, ana mti99en!","They're hiding something, I'm sure!","هوما مخبّيين شي حاجة, أنا متيقّن!"
1,bayna homa tay7awlo ib9aw mbrrdin.,It's obvious they're trying to keep their cool.,باينا هوما تايحاولو إبقاو مبرّدين.
2,loTilat mabaynach fihom mori7in bzzaf.,the hotels don't seem very comfortable.,لوطيلات مابايناش فيهوم موريحين بزّاف.
3,ghaliban ghayjrriw 3lih mn lkhdma!,he is probably about to be laid off by head of...,غاليبان غايجرّيو عليه من لخدما!
4,Tab3an rah mkta2eb!,of course he's depressive!,طابعان راه مكتاءب!
...,...,...,...
12742,ra deja chet imta 5aykoun lfilm,i've already checked the showtimes,را دجا شت إمتا خايكون لفيلم
12743,nhar l7ed 5aydar flil m3etel,there's a late night screening on saturday,نهار لحد خايدار فليل معتل
12744,ana kangoul nmchiw nt3echaw 9bel,i'm thinking we could grab some dinner before ...,أنا كانڭول نمشيو نتعشاو قبل
12745,blan nadi,that's a great plan,بلان نادي


In [6]:
import spacy
def build_vocabulary(texts, max_size=10000):
    nlp = spacy.load('en_core_web_sm')

    all_tokens = []
    for text in texts:
        tokens = [token.text.lower() for token in nlp(text)]
        all_tokens.extend(tokens)

    # Create vocabulary
    token_counts = {}
    for token in all_tokens:
        token_counts[token] = token_counts.get(token, 0) + 1

    # Sort tokens by frequency
    sorted_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)

    # Create vocabulary dictionary
    vocab = {
        '<pad>': 0,
        '<unk>': 1,
        '<sos>': 2,
        '<eos>': 3
    }

    # Add most frequent tokens
    for token, _ in sorted_tokens[:]:
        if token not in vocab:
            vocab[token] = len(vocab)

    return vocab



In [9]:
import unicodedata
# from nltk.translate.bleu_score import sentence_bleu
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import unicodedata
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import spacy
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc="Evaluating", unit="batch")
        for source, target in progress_bar:
            source = source.to(device)
            target = target.to(device)

            output = model(source, target[:, :-1])

            loss = criterion(
                output.contiguous().view(-1, output.size(-1)),
                target[:, 1:].contiguous().view(-1)
            )

            total_loss += loss.item()
            progress_bar.set_postfix({'loss': loss.item()})

    return total_loss / len(dataloader)

def plot_training_history(train_losses, val_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

def train_translation_model(
    lstm_type='vanilla',
    hidden_size=256,
    learning_rate=0.01,
    batch_size=64,
    epochs=50
):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    (train_english, train_darija), (test_english, test_darija) = prepare_dataset()

    source_vocab = build_vocabulary(train_english)
    target_vocab = build_vocabulary(train_darija)

    train_dataset = LanguageDataset(
        train_english,
        train_darija,
        source_vocab,
        target_vocab
    )
    test_dataset = LanguageDataset(
        test_english,
        test_darija,
        source_vocab,
        target_vocab
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn
    )

    translator = DarijaTranslator(
        source_vocab_size=len(source_vocab),
        target_vocab_size=len(target_vocab),
        hidden_size=hidden_size,
        lstm_type=lstm_type
    ).to(device)

    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = optim.Adam(translator.parameters(), lr=learning_rate)

    train_losses = []
    val_losses = []

    os.makedirs('results', exist_ok=True)

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        train_loss = train_epoch(translator, train_loader, criterion, optimizer, device)
        train_losses.append(train_loss)

        val_loss = evaluate(translator, test_loader, criterion, device)
        val_losses.append(val_loss)

        print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

        torch.save({
            'epoch': epoch,
            'model_state_dict': translator.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss
        }, f'results/translation_model_{lstm_type}_epoch_{epoch+1}.pth')

    plot_training_history(train_losses, val_losses)

    return translator, train_losses, val_losses

def compare_lstm_variants():
    lstm_types = ['vanilla', 'peephole', 'working_memory']
    results = {}

    for lstm_type in lstm_types:
        print(f"\n--- Training {lstm_type.upper()} LSTM ---")
        model, train_losses, val_losses = train_translation_model(
            lstm_type=lstm_type,
            hidden_size=256,
            learning_rate=0.001,
            batch_size=64,
            epochs=20
        )

        results[lstm_type] = {
            'model': model,
            'train_losses': train_losses,
            'val_losses': val_losses
        }

    return results

In [10]:


class LanguageDataset(Dataset):
    def __init__(self, source_texts, target_texts, source_vocab, target_vocab):
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.source_vocab = source_vocab
        self.target_vocab = target_vocab


        self.nlp = spacy.load('en_core_web_sm')
        # Tokenizers
        self.source_tokenizer = lambda x: [token.text.lower() for token in self.nlp(x)]
        self.target_tokenizer = lambda x: x.lower().split()

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        # Tokenize and convert to indices
        source_tokens = [self.source_vocab['<sos>']] + \
                        [self.source_vocab.get(token, self.source_vocab['<unk>']) for token in self.source_tokenizer(self.source_texts[idx])] + \
                        [self.source_vocab['<eos>']]

        # For target tokens, use get() method with <unk> as default
        target_tokens = [self.target_vocab['<sos>']] + \
                        [self.target_vocab.get(token, self.target_vocab['<unk>']) for token in self.target_tokenizer(self.target_texts[idx])] + \
                        [self.target_vocab['<eos>']]

        return (torch.tensor(source_tokens), torch.tensor(target_tokens))

def collate_fn(batch):
    # Pad sequences
    source_sequences = [item[0] for item in batch]
    target_sequences = [item[1] for item in batch]

    source_padded = pad_sequence(source_sequences, batch_first=True, padding_value=0)
    target_padded = pad_sequence(target_sequences, batch_first=True, padding_value=0)

    return source_padded, target_padded

def preprocess_text(text):
    # Unicode normalization and lowercasing
    text = unicodedata.normalize('NFKD', text.lower())
    return text

class LSTMBase(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def _init_hidden(self, batch_size, device):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))

class VanillaLSTM(LSTMBase):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.1):
        super().__init__(input_size, hidden_size, num_layers, dropout)

        self.Wxi = nn.Linear(input_size, hidden_size)
        self.Whi = nn.Linear(hidden_size, hidden_size)

        self.Wxf = nn.Linear(input_size, hidden_size)
        self.Whf = nn.Linear(hidden_size, hidden_size)

        self.Wxo = nn.Linear(input_size, hidden_size)
        self.Who = nn.Linear(hidden_size, hidden_size)

        self.Wxg = nn.Linear(input_size, hidden_size)
        self.Whg = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, hidden=None):
        batch_size, seq_len, _ = x.size()
        device = x.device

        if hidden is None:
            h, c = self._init_hidden(batch_size, device)
        else:
            h, c = hidden

        outputs = []
        for t in range(seq_len):
            xt = x[:, t, :]

            it = torch.sigmoid(self.Wxi(xt) + self.Whi(h[-1]))

            ft = torch.sigmoid(self.Wxf(xt) + self.Whf(h[-1]))

            ot = torch.sigmoid(self.Wxo(xt) + self.Who(h[-1]))

            gt = torch.tanh(self.Wxg(xt) + self.Whg(h[-1]))

            c = ft * c + it * gt

            h = ot * torch.tanh(c)

            outputs.append(h)

        return torch.stack(outputs, dim=1), (h, c)

class PeepholeLSTM(LSTMBase):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.1):
        super().__init__(input_size, hidden_size, num_layers, dropout)

        self.Wxi = nn.Linear(input_size, hidden_size)
        self.Whi = nn.Linear(hidden_size, hidden_size)
        self.Wci = nn.Linear(hidden_size, hidden_size)

        self.Wxf = nn.Linear(input_size, hidden_size)
        self.Whf = nn.Linear(hidden_size, hidden_size)
        self.Wcf = nn.Linear(hidden_size, hidden_size)

        self.Wxo = nn.Linear(input_size, hidden_size)
        self.Who = nn.Linear(hidden_size, hidden_size)
        self.Wco = nn.Linear(hidden_size, hidden_size)

        self.Wxg = nn.Linear(input_size, hidden_size)
        self.Whg = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, hidden=None):
        batch_size, seq_len, _ = x.size()
        device = x.device

        if hidden is None:
            h, c = self._init_hidden(batch_size, device)
        else:
            h, c = hidden

        outputs = []
        for t in range(seq_len):
            xt = x[:, t, :]

            it = torch.sigmoid(self.Wxi(xt) + self.Whi(h[-1]) + self.Wci(c))

            ft = torch.sigmoid(self.Wxf(xt) + self.Whf(h[-1]) + self.Wcf(c))

            ot = torch.sigmoid(self.Wxo(xt) + self.Who(h[-1]) + self.Wco(c))

            gt = torch.tanh(self.Wxg(xt) + self.Whg(h[-1]))

            c = ft * c + it * gt

            h = ot * torch.tanh(c)

            outputs.append(h)

        return torch.stack(outputs, dim=1), (h, c)

class WorkingMemoryLSTM(LSTMBase):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.1):
        super().__init__(input_size, hidden_size, num_layers, dropout)

        self.Wxi = nn.Linear(input_size, hidden_size)
        self.Whi = nn.Linear(hidden_size, hidden_size)
        self.Wci = nn.Linear(hidden_size, hidden_size)

        self.Wxf = nn.Linear(input_size, hidden_size)
        self.Whf = nn.Linear(hidden_size, hidden_size)
        self.Wcf = nn.Linear(hidden_size, hidden_size)

        self.Wxo = nn.Linear(input_size, hidden_size)
        self.Who = nn.Linear(hidden_size, hidden_size)
        self.Wco = nn.Linear(hidden_size, hidden_size)

        self.Wxg = nn.Linear(input_size, hidden_size)
        self.Whg = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, hidden=None):
        batch_size, seq_len, _ = x.size()
        device = x.device

        if hidden is None:
            h, c = self._init_hidden(batch_size, device)
        else:
            h, c = hidden

        outputs = []
        for t in range(seq_len):
            xt = x[:, t, :]

            it = torch.sigmoid(self.Wxi(xt) + self.Whi(h[-1]) + torch.tanh(self.Wci(c)))

            ft = torch.sigmoid(self.Wxf(xt) + self.Whf(h[-1]) + torch.tanh(self.Wcf(c)))

            ot = torch.sigmoid(self.Wxo(xt) + self.Who(h[-1]) + torch.tanh(self.Wco(c)))

            gt = torch.tanh(self.Wxg(xt) + self.Whg(h[-1]))

            c = ft * c + it * gt

            h = ot * torch.tanh(c)

            outputs.append(h)

        return torch.stack(outputs, dim=1), (h, c)

class DarijaTranslator(nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, hidden_size, lstm_type='vanilla'):
        super().__init__()

        # Embeddings
        self.source_embedding = nn.Embedding(source_vocab_size, hidden_size)
        self.target_embedding = nn.Embedding(target_vocab_size, hidden_size)

        # Choose LSTM type
        if lstm_type == 'vanilla':
            self.encoder = VanillaLSTM(hidden_size, hidden_size)
            self.decoder = VanillaLSTM(hidden_size, hidden_size)
        elif lstm_type == 'peephole':
            self.encoder = PeepholeLSTM(hidden_size, hidden_size)
            self.decoder = PeepholeLSTM(hidden_size, hidden_size)
        elif lstm_type == 'working_memory':
            self.encoder = WorkingMemoryLSTM(hidden_size, hidden_size)
            self.decoder = WorkingMemoryLSTM(hidden_size, hidden_size)

        self.output_layer = nn.Linear(hidden_size, target_vocab_size)

    def forward(self, source, target):
        source_embedded = self.source_embedding(source)
        target_embedded = self.target_embedding(target)

        encoder_outputs, (encoder_hidden, encoder_cell) = self.encoder(source_embedded)

        decoder_outputs, _ = self.decoder(target_embedded, hidden=(encoder_hidden, encoder_cell))

        output = self.output_layer(decoder_outputs)

        return output

def prepare_dataset():
    dataset = load_dataset("imomayiz/darija-english", 'sentences')

    dataset.set_format(type='pandas', output_all_columns=True)
    dataset = dataset['sentences'][:]
    df = dataset.dropna()
    df = df.rename(columns={'eng': 'english'})
    df['english'] = df['english'].apply(preprocess_text)
    df['darija'] = df['darija'].apply(preprocess_text)

    english_sentences = df['english'].tolist()
    darija_sentences = df['darija'].tolist()

    train_english, test_english, train_darija, test_darija = train_test_split(
        english_sentences,
        darija_sentences,
        test_size=0.2,
        random_state=54
    )

    return (train_english, train_darija), (test_english, test_darija)

def build_vocabulary(texts, max_size=10000):
    import spacy
    nlp = spacy.load('en_core_web_sm')

    all_tokens = []
    for text in texts:
        tokens = [token.text.lower() for token in nlp(text)]
        all_tokens.extend(tokens)

    token_counts = {}
    for token in all_tokens:
        token_counts[token] = token_counts.get(token, 0) + 1

    sorted_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)

    vocab = {
        '<pad>': 0,
        '<unk>': 1,
        '<sos>': 2,
        '<eos>': 3
    }

    for token, _ in sorted_tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

    return vocab

def train_epoch(model, dataloader, criterion, optimizer, device='cuda'):
    model.train()
    total_loss = 0

    progress_bar = tqdm(dataloader, desc="Training", unit="batch")
    for source, target in progress_bar:
        source = source.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        # Note: For simplicity, we're using teacher forcing
        output = model(source, target[:, :-1])

        # Reshape for loss calculation
        loss = criterion(
            output.contiguous().view(-1, output.size(-1)),
            target[:, 1:].contiguous().view(-1)
        )

        # Backward pass
        loss.backward()

        # Clip gradients to prevent exploding gradient problem, since we're using 0 dropout or weight decay
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()

        total_loss += loss.item()

        progress_bar.set_postfix({'loss': loss.item()})

    return total_loss / len(dataloader)




results = compare_lstm_variants()


--- Training VANILLA LSTM ---
Using device: cuda

Epoch 1/20


Evaluating: 100%|██████████| 40/40 [00:17<00:00,  2.29batch/s, loss=5.64]


Train Loss: 7.0393, Validation Loss: 5.4473

Epoch 2/20


Evaluating: 100%|██████████| 40/40 [00:16<00:00,  2.40batch/s, loss=5.62]


Train Loss: 6.6193, Validation Loss: 5.4075

Epoch 3/20


...

Train Loss: 6.5561, Validation Loss: 5.4502

Epoch 19/20


Evaluating: 100%|██████████| 40/40 [00:17<00:00,  2.31batch/s, loss=5.58]


Train Loss: 6.5549, Validation Loss: 5.4070

Epoch 20/20


Evaluating: 100%|██████████| 40/40 [00:17<00:00,  2.33batch/s, loss=5.62]


Train Loss: 6.5516, Validation Loss: 5.4318


Huge loss, too lost in code to figure out if I'm not defining the pipeline well, or if this is a case of insufficient data, or a model that is too simple (or a mix of some of these factors).

However we test the models with a random init using 0s and the normal distribution, as we're already using Xavier's

In [13]:
def translate_sentence(model, sentence, source_vocab, target_vocab, device):
    model.eval()

    sentence = preprocess_text(sentence)


    nlp = spacy.load('en_core_web_sm')
    source_tokens = [source_vocab['<sos>']] + \
                    [source_vocab.get(token.text.lower(), source_vocab['<unk>'])
                     for token in nlp(sentence)] + \
                    [source_vocab['<eos>']]
    source_tensor = torch.tensor(source_tokens).unsqueeze(0).to(device)  # Add batch dimension


    with torch.no_grad():  # No need to track gradients during inference
        encoder_outputs, (encoder_hidden, encoder_cell) = model.encoder(model.source_embedding(source_tensor))

        decoder_input = torch.tensor([[target_vocab['<sos>']]]).to(device)

        translated_tokens = []

        for _ in range(50):
            decoder_output, (decoder_hidden, decoder_cell) = model.decoder(model.target_embedding(decoder_input), (encoder_hidden, encoder_cell))

            predicted_token_index = decoder_output.argmax(dim=-1).item()

            if predicted_token_index == target_vocab['<eos>']:
                break

            translated_tokens.append(predicted_token_index)

            decoder_input = torch.tensor([[predicted_token_index]]).to(device)
            encoder_hidden, encoder_cell = decoder_hidden, decoder_cell

    translated_sentence = " ".join([
        word for word, index in target_vocab.items()
        if index in translated_tokens
    ])

    return translated_sentence

example_sentence = "This is a test sentence."
lstm_type = "peephole"
model = results[lstm_type]['model']
source_vocab = build_vocabulary(prepare_dataset()[0][0])
target_vocab = build_vocabulary(prepare_dataset()[0][1])
translated_sentence = translate_sentence(model, example_sentence, source_vocab, target_vocab, device='cuda')
print(f"Original sentence: {example_sentence}")
print(f"Translated sentence: {translated_sentence}")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Original sentence: This is a test sentence.
Translated sentence: wach ana rah daba


In [11]:
results

{'vanilla': {'model': DarijaTranslator(
    (source_embedding): Embedding(4944, 256)
    (target_embedding): Embedding(12970, 256)
    (encoder): VanillaLSTM(
      (Wxi): Linear(in_features=256, out_features=256, bias=True)
      (Whi): Linear(in_features=256, out_features=256, bias=True)
      (Wxf): Linear(in_features=256, out_features=256, bias=True)
      (Whf): Linear(in_features=256, out_features=256, bias=True)
      (Wxo): Linear(in_features=256, out_features=256, bias=True)
      (Who): Linear(in_features=256, out_features=256, bias=True)
      (Wxg): Linear(in_features=256, out_features=256, bias=True)
      (Whg): Linear(in_features=256, out_features=256, bias=True)
    )
    (decoder): VanillaLSTM(
      (Wxi): Linear(in_features=256, out_features=256, bias=True)
      (Whi): Linear(in_features=256, out_features=256, bias=True)
      (Wxf): Linear(in_features=256, out_features=256, bias=True)
      (Whf): Linear(in_features=256, out_features=256, bias=True)
      (Wxo): Li

In [14]:
example_sentence = "We didn't do well in this project"
for lstm_type in ['vanilla', 'peephole', 'working_memory']:
    translated_sentence = translate_sentence(model, example_sentence, source_vocab, target_vocab, device='cuda')
    print(f"Translated sentence with {lstm_type}: {translated_sentence}")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Translated sentence with vanilla: . wach ana rah daba
Translated sentence with peephole: . wach ana rah daba
Translated sentence with working_memory: . wach ana rah daba


In [15]:
example_sentence = "I was lost and couldn't find the shop."
for lstm_type in ['vanilla', 'peephole', 'working_memory']:
    translated_sentence = translate_sentence(model, example_sentence, source_vocab, target_vocab, device='cuda')
    print(f"Translated sentence with {lstm_type}: {translated_sentence}")

Translated sentence with vanilla: . wach ana rah daba
Translated sentence with peephole: . wach ana rah daba
Translated sentence with working_memory: . wach ana rah daba


In [18]:
example_sentence = "I am sure."
for lstm_type in ['vanilla', 'peephole', 'working_memory']:
    translated_sentence = translate_sentence(model, example_sentence, source_vocab, target_vocab, device='cuda')
    print(f"Translated sentence with {lstm_type}: {translated_sentence}")

Translated sentence with vanilla: . wach ana rah daba dyalk ou
Translated sentence with peephole: . wach ana rah daba dyalk ou
Translated sentence with working_memory: . wach ana rah daba dyalk ou


In [20]:
example_sentence = "hi"
for lstm_type in ['vanilla', 'peephole', 'working_memory']:
    translated_sentence = translate_sentence(model, example_sentence, source_vocab, target_vocab, device='cuda')
    print(f"Translated sentence with {lstm_type}: {translated_sentence}")

Translated sentence with vanilla: wach ana rah daba dyalk ou a liyya
Translated sentence with peephole: wach ana rah daba dyalk ou a liyya
Translated sentence with working_memory: wach ana rah daba dyalk ou a liyya


Absolutely horrendous, it brings to mind n-gram models.

Testing more cells isn't an option since I ran out of time.